# Run this to create a CSV of all fighters

You may need need to change max_workers to a lower value depending on your CPU.

In [3]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import csv
import concurrent.futures
import time
import random
import pandas as pd

alphabet = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
all_fighter_info = []
fighter_id = 0

def fetch_fighter_links(char):
    url = f"http://ufcstats.com/statistics/fighters?char={char}&page=all"
    while True:
        try:
            page = urlopen(url)
            html = page.read().decode("utf-8")
            soup = BeautifulSoup(html, "html.parser")
            links = []
            for fighter_alpha in soup.find_all("tr", class_="b-statistics__table-row")[2:]:
                fighter = fighter_alpha.find("a")
                if fighter:
                    links.append(fighter.get("href"))
            print(f"Fetched {len(links)} links for fighters with last names starting with '{char}'")
            return links
        except Exception as e:
            print(f"Error fetching links for {char}: {e}")
            time.sleep(random.uniform(2, 3))  # Wait before retrying

def fetch_fighter_info(fighter_info):
    global fighter_id
    fighter_link, index = fighter_info
    while True:
        try:
            page = urlopen(fighter_link)
            html = page.read().decode("utf-8")
            parse = BeautifulSoup(html, "html.parser")

            fighter_id += 1
            name = parse.find("span", "b-content__title-highlight").text.strip()
            nickname = parse.find("p", "b-content__Nickname").text.strip()

            information = parse.find_all("li", class_="b-list__box-list-item_type_block")
            height = information[0].i.next_sibling.text.strip().replace("'", "").replace("\"", "").split()
            height = (int(height[0]) * 12 + int(height[1])) if height != ["--"] else ""

            reach = information[2].i.next_sibling.text.strip().replace("'", "").replace("\"", "")
            reach = int(reach) if reach and reach[0] != "-" else ""

            weight = information[1].i.next_sibling.text.strip().replace("'", "").replace("\"", "")[:-5]
            weight = int(weight) if weight else ""

            stance = information[3].i.next_sibling.text.strip() or ""

            dob_info = information[4].i.next_sibling.text.strip()
            dob = ""
            if dob_info and dob_info[0] != "-":
                dob_parts = dob_info.replace('\n', '').strip().split()
                year, month, day = dob_parts[2], dob_parts[0], dob_parts[1][:-1]
                month_map = {
                    "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04",
                    "May": "05", "Jun": "06", "Jul": "07", "Aug": "08",
                    "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"
                }
                month = month_map.get(month, "00")
                dob = f"{year}-{month}-{day}"

            stats = [
                float(information[5].i.next_sibling.text.strip()),
                int(information[6].i.next_sibling.text.strip()[:-1]) * 0.01,
                float(information[7].i.next_sibling.text.strip()),
                int(information[8].i.next_sibling.text.strip()[:-1]) * 0.01,
                float(information[10].i.next_sibling.text.strip()),
                int(information[11].i.next_sibling.text.strip()[:-1]) * 0.01,
                int(information[12].i.next_sibling.text.strip()[:-1]) * 0.01,
                float(information[13].i.next_sibling.text.strip()),
            ]

            fighter_info = [fighter_id, name, nickname, height, reach, weight, stance, dob] + stats + [fighter_link]
            print(f"Fetched fighter info: fetch ({fighter_id})")
            return (index, fighter_info)  # Return index along with fighter info
        except Exception as e:
            print(f"Error fetching info for {fighter_link}: {e}")
            time.sleep(random.uniform(2, 3))  # Wait before retrying

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    all_fighter_links = list(executor.map(fetch_fighter_links, alphabet))

all_fighter_links = [link for sublist in all_fighter_links for link in sublist]

fighter_infos_with_index = [(link, i) for i, link in enumerate(all_fighter_links)]

with open('fighters.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    field = ["fighter_id", "name", "nickname", "height", "reach", "weight", "stance", "dob",
             "slpm", "str_acc", "sapm", "str_def", "td_avg", "td_acc", "td_def", "sub_avg", "fighter_link"]
    writer.writerow(field)

    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        fighter_infos_with_index = list(executor.map(fetch_fighter_info, fighter_infos_with_index))
        
    fighter_infos_with_index.sort(key=lambda x: x[0])  # Sort by original index
    fighter_infos = [info for _, info in fighter_infos_with_index]  # Extract fighter info

    writer.writerows(fighter_infos)

df = pd.read_csv("fighters.csv")
df['fighter_id'] = range(1, len(df) + 1)
df.to_csv("fighters.csv", index=False)

print(f"Total fighters processed: {len(df)}")

Fetched 223 links for fighters with last names starting with 'a'
Fetched 45 links for fighters with last names starting with 'i'
Fetched 139 links for fighters with last names starting with 'f'
Fetched 201 links for fighters with last names starting with 'd'
Fetched 308 links for fighters with last names starting with 'b'
Fetched 92 links for fighters with last names starting with 'e'
Fetched 123 links for fighters with last names starting with 'j'
Fetched 222 links for fighters with last names starting with 'h'
Fetched 229 links for fighters with last names starting with 'g'
Fetched 203 links for fighters with last names starting with 'l'
Fetched 281 links for fighters with last names starting with 'c'
Fetched 80 links for fighters with last names starting with 'o'
Fetched 171 links for fighters with last names starting with 'k'
Fetched 22 links for fighters with last names starting with 'u'
Fetched 13 links for fighters with last names starting with 'q'
Fetched 113 links for fighters

# Run this to create a CSV of all events

In [17]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup
import concurrent.futures
import time
import random
import pandas as pd

def fetch_event_info(event_counter):
    url = "http://ufcstats.com/statistics/events/completed?page=all"
    
    while True:
        try:
            page = urlopen(url)
            html = page.read().decode("utf-8")
            soup = BeautifulSoup(html, "html.parser")

            events = soup.find_all("tr", class_="b-statistics__table-row")[event_counter]
            event_name = events.find("a").text.strip()
            event_link = events.find("a").get("href")
            date = events.find("span").text.strip()
            location = events.find_all("td")[-1].text.strip()

            date_parts = date.split()
            year = date_parts[2]
            day = date_parts[1][:-1]
            month = date_parts[0]
            month_map = {
                "January": "01", "February": "02", "March": "03",
                "April": "04", "May": "05", "June": "06",
                "July": "07", "August": "08", "September": "09",
                "October": "10", "November": "11", "December": "12"
            }
            month = month_map.get(month, "00")
            date = f"{year}-{month}-{day}"

            location_parts = location.split(", ")
            if len(location_parts) == 2:
                city = location_parts[0]
                region = location_parts[0]
                nation = location_parts[1]
            elif len(location_parts) == 3:
                city = location_parts[0]
                region = location_parts[1]
                nation = location_parts[2]

            return (event_counter, [event_counter, event_name, date, city, region, nation, event_link])

        except Exception as e:
            print(f"Error fetching info for event {event_counter}: {e}")
            time.sleep(random.uniform(2, 5))

url = "http://ufcstats.com/statistics/events/completed?page=all"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

event_len = len(soup.find_all("tr", class_="b-statistics__table-row"))
all_event_info = [None] * (event_len - 2)  # Preallocate the list

with open('events.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    field = ["event_id", "event_name", "date", "city", "region", "nation", "event_link"]
    writer.writerow(field)

    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        futures = {executor.submit(fetch_event_info, i): i for i in range(2, event_len)}

        for future in concurrent.futures.as_completed(futures):
            try:
                index, event_info = future.result()  # Get index and event info
                all_event_info[index] = event_info  # Place it in the correct order
                print(f"Processed event: {event_info[0]}/{event_len - 2}")
            except Exception as e:
                print(f"Error processing event: {e}")

    # Write all event info to the CSV after fetching
    for event_info in all_event_info:
        if event_info is not None:
            writer.writerow(event_info)

df = pd.read_csv("events.csv")
df['event_id'] = range(1, len(df) + 1)
df.to_csv("events.csv", index=False)

print(f"Total events processed: {len([info for info in all_event_info if info])}")


Error fetching info for event 8: HTTP Error 429: Too Many RequestsError fetching info for event 15: HTTP Error 429: Too Many Requests
Error fetching info for event 14: HTTP Error 429: Too Many Requests
Error fetching info for event 12: HTTP Error 429: Too Many Requests
Error fetching info for event 18: HTTP Error 429: Too Many Requests
Error fetching info for event 20: HTTP Error 429: Too Many Requests
Error fetching info for event 16: HTTP Error 429: Too Many Requests
Error fetching info for event 17: HTTP Error 429: Too Many Requests

Error fetching info for event 19: HTTP Error 429: Too Many Requests
Error fetching info for event 21: HTTP Error 429: Too Many Requests
Processed event: 10/708
Processed event: 6/708
Processed event: 3/708
Processed event: 2/708
Processed event: 9/708
Processed event: 4/708
Processed event: 23/708
Processed event: 7/708
Processed event: 22/708
Processed event: 11/708
Processed event: 24/708
Processed event: 5/708
Processed event: 21/708
Processed event:

# Run this to create a CSV of fights

In [3]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup
import concurrent.futures
import time
import random
import threading
import pandas as pd

# Global stop flag
stop_flag = threading.Event()

def fetch_fight_details(event_info):
    event_link, event_name = event_info
    fight_details = []

    while not stop_flag.is_set():  # Check the stop flag
        try:
            page = urlopen(event_link)
            html = page.read().decode("utf-8")
            soup = BeautifulSoup(html, "html.parser")
            fights = soup.find_all(class_='b-fight-details__table-row')[1:]

            for fight in fights:
                if stop_flag.is_set():  # Check the stop flag inside the loop
                    return fight_details  # Exit if the stop flag is set
                
                fight_link = fight["data-link"]
                fight_page = urlopen(fight_link)
                fight_html = fight_page.read().decode("utf-8")
                fight_soup = BeautifulSoup(fight_html, "html.parser")

                fighter_1 = fight_soup.find_all(class_='b-fight-details__person-link')[0].text.strip()
                fighter_1_link = fight_soup.find_all(class_='b-fight-details__person-link')[0].get("href")
                fighter_1_result = fight_soup.find_all(class_="b-fight-details__person-status")[0].text.strip()

                fighter_2 = fight_soup.find_all(class_='b-fight-details__person-link')[1].text.strip()
                fighter_2_link = fight_soup.find_all(class_='b-fight-details__person-link')[1].get("href")
                fighter_2_result = fight_soup.find_all(class_="b-fight-details__person-status")[1].text.strip()

                weight_class = fight_soup.find_all(class_='b-fight-details__fight-title')[0].text.strip()
                method = fight_soup.find_all(class_='b-fight-details__label')[0].find_next_sibling().text.strip()

                info = fight_soup.find_all(class_='b-fight-details__text-item')
                end_round = info[0].text.strip()[-1]
                end_time_round = info[1].text.strip()[-4:]
                end_time_round = int(end_time_round.split(":")[0]) * 60 + int(end_time_round.split(":")[1])
                max_round = info[2].text.strip()[12:].strip()[0] if info[2].text.strip()[12:] else ""
                referee = info[3].text.strip()[20:].strip()

                fight_details.append([event_name, fighter_1_link, fighter_1, fighter_1_result,
                                      fighter_2_link, fighter_2, fighter_2_result, weight_class,
                                      method, end_round, end_time_round, max_round, referee, fight_link])

            return fight_details  # Return if successful

        except Exception as e:
            if stop_flag.is_set():  # Exit on stop flag
                return fight_details
            print(f"Error fetching fight details for {event_name}: {e}")
            time.sleep(random.uniform(2, 5))  # Wait before retrying

# Read events from CSV
all_events_link = []
all_events_name = []

with open('events.csv', mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        all_events_link.append(row[6])
        all_events_name.append(row[1])

event_name_and_link = list(zip(all_events_link[1:], all_events_name[1:]))

fight_id = 0
processed = 0

with open('fights_overview.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    field = ["fight_id", "event_name", "fighter_1_link", "fighter_1", "fighter_1_result",
             "fighter_2_link", "fighter_2", "fighter_2_result", "bout_type", "method", 
             "end_round", "end_time_round", "max_round", "referee", "fight_link"]
    writer.writerow(field)

    futures = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        for event_info in event_name_and_link:
            future = executor.submit(fetch_fight_details, event_info)
            futures.append(future)

        # Process futures in order
        for future in futures:
            fight_details = future.result()
            for detail in fight_details:
                if stop_flag.is_set():  # Check stop flag before writing
                    break
                fight_id += 1
                processed += 1
                writer.writerow([fight_id] + detail)
                print(f"Processed fight: {processed}")

print(f"Total fights processed: {processed}")


Processed fight: 1
Processed fight: 2
Processed fight: 3
Processed fight: 4
Processed fight: 5
Processed fight: 6
Processed fight: 7
Processed fight: 8
Processed fight: 9
Processed fight: 10
Processed fight: 11
Processed fight: 12
Processed fight: 13
Processed fight: 14
Processed fight: 15
Processed fight: 16
Processed fight: 17
Processed fight: 18
Processed fight: 19
Processed fight: 20
Processed fight: 21
Processed fight: 22
Processed fight: 23
Processed fight: 24
Processed fight: 25
Processed fight: 26
Processed fight: 27
Processed fight: 28
Processed fight: 29
Processed fight: 30
Processed fight: 31
Processed fight: 32
Processed fight: 33
Processed fight: 34
Processed fight: 35
Processed fight: 36
Processed fight: 37
Processed fight: 38
Processed fight: 39
Processed fight: 40
Processed fight: 41
Processed fight: 42
Processed fight: 43
Processed fight: 44
Processed fight: 45
Processed fight: 46
Processed fight: 47
Processed fight: 48
Processed fight: 49
Processed fight: 50
Processed

# Run this to create a CSV of all rounds

In [17]:
import time
import csv
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock
import pandas as pd

def scrape_fight(fight):
    round_data = []
    round_id = 0
    current_round = 0

    while True:  # Keep trying until successful
        try:
            page = urlopen(fight)
            break  # Exit the retry loop if successful
        except HTTPError as e:
            if e.code == 429:
                print("Rate limit hit. Waiting before retrying...")
                time.sleep(5)  # Wait before retrying
            else:
                print(f"HTTP error: {e}")
                return []  # Return an empty list for other HTTP errors

    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    rounds = soup.find_all("tr", class_="b-fight-details__table-row")
    all_rounds = int((len(rounds) - 6) / 2)
    round_index = list(range(3, 3 + all_rounds)) + list(range(7 + all_rounds, 7 + all_rounds + all_rounds))
    first_half = round_index[:len(round_index) // 2]
    second_half = round_index[len(round_index) // 2:]
    round_index = list(zip(first_half, second_half))

    for round in round_index:
        current_round += 1
        fighter_1 = []
        fighter_2 = []

        names = soup.find_all("a", class_="b-link")
        fight_name = names[0].text.strip()
        f_1_name = names[1].text.strip()
        fighter_1_link = names[1].get("href")
        f_2_name = names[2].text.strip()
        fighter_2_link = names[2].get("href")

        fight_stats_top = rounds[round[0]].find_all("p", "b-fight-details__table-text")
        fight_stats_bottom = rounds[round[1] - 1].find_all("p", "b-fight-details__table-text")

        # Extract stats for fighter 1
        f_1_kd = fight_stats_top[2].text.strip()
        f_2_kd = fight_stats_top[3].text.strip()

        f_1_ts_landed = fight_stats_top[8].text.strip().split(" ")[0]
        f_1_ts_attempted = fight_stats_top[8].text.strip().split(" ")[2]
        f_2_ts_landed = fight_stats_top[9].text.strip().split(" ")[0]
        f_2_ts_attempted = fight_stats_top[9].text.strip().split(" ")[2]

        f_1_td_landed = fight_stats_top[10].text.strip().split(" ")[0]
        f_1_td_attempted = fight_stats_top[10].text.strip().split(" ")[2]
        f_2_td_landed = fight_stats_top[11].text.strip().split(" ")[0]
        f_2_td_attempted = fight_stats_top[11].text.strip().split(" ")[2]

        f_1_sub_attempted = fight_stats_top[14].text.strip()
        f_2_sub_attempted = fight_stats_top[15].text.strip()

        f_1_rev = fight_stats_top[16].text.strip()
        f_2_rev = fight_stats_top[17].text.strip()

        f_1_control = fight_stats_top[18].text.strip().split(":")
        f_1_control = "" if f_1_control == ["--"] else int(f_1_control[0]) * 60 + int(f_1_control[1])
        f_2_control = fight_stats_top[19].text.strip().split(":")
        f_2_control = "" if f_2_control == ["--"] else int(f_2_control[0]) * 60 + int(f_2_control[1])

        f_1_ss_landed = fight_stats_bottom[2].text.strip().split(" ")[0]
        f_1_ss_attempted = fight_stats_bottom[2].text.strip().split(" ")[2]
        f_2_ss_landed = fight_stats_bottom[3].text.strip().split(" ")[0]
        f_2_ss_attempted = fight_stats_bottom[3].text.strip().split(" ")[2]

        f_1_head_ss_attempted = fight_stats_bottom[6].text.strip().split(" ")[0]
        f_1_head_ss_landed = fight_stats_bottom[6].text.strip().split(" ")[2]
        f_2_head_ss_attempted = fight_stats_bottom[7].text.strip().split(" ")[0]
        f_2_head_ss_landed = fight_stats_bottom[7].text.strip().split(" ")[2]

        f_1_body_ss_attempted = fight_stats_bottom[8].text.strip().split(" ")[0]
        f_1_body_ss_landed = fight_stats_bottom[8].text.strip().split(" ")[2]
        f_2_body_ss_attempted = fight_stats_bottom[9].text.strip().split(" ")[0]
        f_2_body_ss_landed = fight_stats_bottom[9].text.strip().split(" ")[2]

        f_1_leg_ss_attempted = fight_stats_bottom[10].text.strip().split(" ")[0]
        f_1_leg_ss_landed = fight_stats_bottom[10].text.strip().split(" ")[2]
        f_2_leg_ss_attempted = fight_stats_bottom[11].text.strip().split(" ")[0]
        f_2_leg_ss_landed = fight_stats_bottom[11].text.strip().split(" ")[2]

        f_1_distance_ss_attempted = fight_stats_bottom[12].text.strip().split(" ")[0]
        f_1_distance_ss_landed = fight_stats_bottom[12].text.strip().split(" ")[2]
        f_2_distance_ss_attempted = fight_stats_bottom[13].text.strip().split(" ")[0]
        f_2_distance_ss_landed = fight_stats_bottom[13].text.strip().split(" ")[2]

        f_1_clinch_attempted = fight_stats_bottom[14].text.strip().split(" ")[0]
        f_1_clinch_landed = fight_stats_bottom[14].text.strip().split(" ")[2]
        f_2_clinch_attempted = fight_stats_bottom[15].text.strip().split(" ")[0]
        f_2_clinch_landed = fight_stats_bottom[15].text.strip().split(" ")[2]

        f_1_ground_attempted = fight_stats_bottom[16].text.strip().split(" ")[0]
        f_1_ground_landed = fight_stats_bottom[16].text.strip().split(" ")[2]
        f_2_ground_attempted = fight_stats_bottom[17].text.strip().split(" ")[0]
        f_2_ground_landed = fight_stats_bottom[17].text.strip().split(" ")[2]

        round_id += 1

        fighter_1 = [round_id, fight_name, current_round, fighter_1_link, f_1_name, f_1_kd, f_1_ts_landed, f_1_ts_attempted,
                     f_1_td_landed, f_1_td_attempted, f_1_sub_attempted, f_1_rev, f_1_control, f_1_ss_landed,
                     f_1_ss_attempted, f_1_head_ss_attempted, f_1_head_ss_landed, f_1_body_ss_attempted,
                     f_1_body_ss_landed, f_1_leg_ss_attempted, f_1_leg_ss_landed, f_1_distance_ss_attempted,
                     f_1_distance_ss_landed, f_1_clinch_attempted, f_1_clinch_landed, f_1_ground_attempted,
                     f_1_ground_landed]

        fighter_2 = [round_id, fight_name, current_round, fighter_2_link, f_2_name, f_2_kd, f_2_ts_landed, f_2_ts_attempted,
                     f_2_td_landed, f_2_td_attempted, f_2_sub_attempted, f_2_rev, f_2_control, f_2_ss_landed,
                     f_2_ss_attempted, f_2_head_ss_attempted, f_2_head_ss_landed, f_2_body_ss_attempted,
                     f_2_body_ss_landed, f_2_leg_ss_attempted, f_2_leg_ss_landed, f_2_distance_ss_attempted,
                     f_2_distance_ss_landed, f_2_clinch_attempted, f_2_clinch_landed, f_2_ground_attempted,
                     f_2_ground_landed]

        round_data.append((fighter_1, fighter_2))

    return round_data

all_fight_links = []

# Read fight links from CSV
with open('fights_overview.csv', 'r', newline='') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        all_fight_links.append(row[14])

all_fight_links = all_fight_links[1:]  # Skip header
num_fights = len(all_fight_links)

# Prepare CSV for writing
lock = Lock()

# Write results to CSV
def write_to_csv(data):
    with lock:
        with open('rounds.csv', 'a', newline='', encoding="utf-8") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(data)

# Prepare CSV header
with open('rounds.csv', 'w', newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    field = ["round_id", "event_name", "round", "fighter_link", "fighter_name", "kd", "ts_landed"]
    field += ["ts_attempted", "td_landed", "td_attempted", "sub_attempted", "rev", "control", "ss_landed"]
    field += ["ss_attempted", "head_ss_attempted", "head_ss_landed", "body_ss_attempted", "body_ss_landed",
              "leg_ss_attempted", "leg_ss_landed", "distance_ss_attempted", "distance_ss_landed",
              "clinch_ss_attempted", "clinch_ss_landed", "ground_ss_attempted", "ground_ss_landed"]
    writer.writerow(field)

with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_fight = {executor.submit(scrape_fight, fight): fight for fight in all_fight_links}
    fight_counter = 0
    for future in as_completed(future_to_fight):
        fight = future_to_fight[future]
        try:
            round_data = future.result()
            for fighter_1, fighter_2 in round_data:
                write_to_csv(fighter_1)
                write_to_csv(fighter_2)
            fight_counter += 1
            print(f"Finished scraping rounds for fight: {fight_counter}/{num_fights}")

        except Exception as e:
            print(f"Error scraping fight {fight}: {e}")

df = pd.read_csv("rounds.csv")
df['round_id'] = range(1, len(df) + 1)
df.to_csv("rounds.csv", index=False)

all_event_links = []

with open('events.csv', 'r', newline='', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        all_event_links .append(row[1])
all_event_links = all_event_links[1:]

df = pd.read_csv('rounds.csv')
df['event_name'] = pd.Categorical(df['event_name'], categories=all_event_links, ordered=True)
df = df.sort_values(by='event_name')
df['round_id'] = range(1, len(df) + 1)

df.to_csv('rounds.csv', index=False)
print("All rounds scraped.")

Finished scraping rounds for fight: 1/7864
Finished scraping rounds for fight: 2/7864
Finished scraping rounds for fight: 3/7864
Finished scraping rounds for fight: 4/7864
Finished scraping rounds for fight: 5/7864
Finished scraping rounds for fight: 6/7864
Finished scraping rounds for fight: 7/7864
Finished scraping rounds for fight: 8/7864
Finished scraping rounds for fight: 9/7864
Finished scraping rounds for fight: 10/7864
Finished scraping rounds for fight: 11/7864
Finished scraping rounds for fight: 12/7864
Finished scraping rounds for fight: 13/7864
Finished scraping rounds for fight: 14/7864
Finished scraping rounds for fight: 15/7864
Finished scraping rounds for fight: 16/7864
Finished scraping rounds for fight: 17/7864
Finished scraping rounds for fight: 18/7864
Finished scraping rounds for fight: 19/7864
Finished scraping rounds for fight: 20/7864
Finished scraping rounds for fight: 21/7864
Finished scraping rounds for fight: 22/7864
Finished scraping rounds for fight: 23/78